# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Init Dataset

In [3]:
#sample dataset
import numpy as np

image_shape = [512, 512]
n_class = 21 #background + 20 label
mask = True
max_pad_size = 100
total_data_size = 100

tr_x = ["a.jpg" for _ in range(total_data_size)]
tr_y = ["a.xml" for _ in range(total_data_size)]

# 2. Pre-load Generator

In [4]:
def load(*args, **kwargs):
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    y_true = np.random.randint(n_class, size = [max_pad_size, 1])
    bbox_true = np.array([tfdet.core.bbox.random_bbox(image_shape = image_shape) for _ in range(max_pad_size)])
    result = [image, y_true, bbox_true]
    if mask:
        mask_true = np.random.random((max_pad_size, *image_shape, 1)).astype(np.float32)
        result = [image, y_true, bbox_true, mask_true]
    return tuple(result)

def generator():
    for x, y in zip(tr_x, tr_y):
        yield load(x, y)
        
dtype = (tf.float32, tf.int32, tf.int32, tf.float32) if mask else (tf.float32, tf.int32, tf.int32)
tr_pipe = tf.data.Dataset.from_generator(generator, dtype)

In [5]:
#tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe) #for train_model to object detection
tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = image_shape, keep_ratio = True)
tr_pipe = tfdet.dataset.pipeline.filter_annotation(tr_pipe, min_scale = 2, min_instance_area = 1)
#tr_pipe = tfdet.dataset.pipeline.label_encode(tr_pipe)
tr_pipe = tfdet.dataset.pipeline.normalize(tr_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375], bbox_normalize = True)
tr_pipe = tfdet.dataset.pipeline.pad(tr_pipe, image_shape = image_shape, max_pad_size = 100, mode = "both")
tr_pipe = tfdet.dataset.pipeline.cast(tr_pipe, map = {"x_true":tf.float32, "y_true":tf.float32, "bbox_true":tf.float32, "mask_true":tf.float32})
tr_pipe = tr_pipe.batch(4).prefetch(1)

In [6]:
out = next(iter(tr_pipe))
image, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

if mask_true is None:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}".format(image.shape, y_true.shape, bbox_true.shape))
else:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}, mask_true:{3}".format(image.shape, y_true.shape, bbox_true.shape, mask_true.shape))

x_true:(4, 512, 512, 3), y_true:(4, 100, 1), bbox_true:(4, 100, 4), mask_true:(4, 100, 512, 512, 1)


# 3. Post-load Generator

In [7]:
def load_image(*args, **kwargs):
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    return image

def load_anno(*args, **kwargs):
    y_true = np.random.randint(n_class, size = [max_pad_size, 1])
    bbox_true = np.array([tfdet.core.bbox.random_bbox(image_shape = image_shape) for _ in range(max_pad_size)])
    result = [y_true, bbox_true]
    if mask:
        mask_true = np.random.random((max_pad_size, *image_shape, 1)).astype(np.float32)
        result = [y_true, bbox_true, mask_true]
    return tuple(result)

def generator():
    for x, y in zip(tr_x, tr_y):
        yield x, y
        
dtype = (tf.string, tf.string)
tr_pipe = tf.data.Dataset.from_generator(generator, dtype)
tr_pipe = tfdet.dataset.pipeline.load(tr_pipe, load_func = load_image, anno_func = load_anno) #post load

In [8]:
#tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe) #for train_model to object detection
tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = image_shape, keep_ratio = True)
tr_pipe = tfdet.dataset.pipeline.filter_annotation(tr_pipe, min_scale = 2, min_instance_area = 1)
#tr_pipe = tfdet.dataset.pipeline.label_encode(tr_pipe)
tr_pipe = tfdet.dataset.pipeline.normalize(tr_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375], bbox_normalize = True)
tr_pipe = tfdet.dataset.pipeline.pad(tr_pipe, image_shape = image_shape, max_pad_size = 100, mode = "both")
tr_pipe = tfdet.dataset.pipeline.cast(tr_pipe, map = {"x_true":tf.float32, "y_true":tf.float32, "bbox_true":tf.float32, "mask_true":tf.float32})
tr_pipe = tr_pipe.batch(4).prefetch(1)

In [9]:
out = next(iter(tr_pipe))
image, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

if mask_true is None:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}".format(image.shape, y_true.shape, bbox_true.shape))
else:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}, mask_true:{3}".format(image.shape, y_true.shape, bbox_true.shape, mask_true.shape))

x_true:(4, 512, 512, 3), y_true:(4, 100, 1), bbox_true:(4, 100, 4), mask_true:(4, 100, 512, 512, 1)


# 4. List

In [10]:
def load_image(*args, **kwargs):
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    return image

def load_anno(*args, **kwargs):
    y_true = np.random.randint(n_class, size = [max_pad_size, 1])
    bbox_true = np.array([tfdet.core.bbox.random_bbox(image_shape = image_shape) for _ in range(max_pad_size)])
    result = [y_true, bbox_true]
    if mask:
        mask_true = np.random.random((max_pad_size, *image_shape, 1)).astype(np.float32)
        result = [y_true, bbox_true, mask_true]
    return tuple(result)

tr_pipe = tfdet.dataset.pipeline.load(tr_x, y_true = tr_y, load_func = load_image, anno_func = load_anno) #post load

In [11]:
#tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe) #for train_model to object detection
tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = image_shape, keep_ratio = True)
tr_pipe = tfdet.dataset.pipeline.filter_annotation(tr_pipe, min_scale = 2, min_instance_area = 1)
#tr_pipe = tfdet.dataset.pipeline.label_encode(tr_pipe)
tr_pipe = tfdet.dataset.pipeline.normalize(tr_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375], bbox_normalize = True)
tr_pipe = tfdet.dataset.pipeline.pad(tr_pipe, image_shape = image_shape, max_pad_size = 100, mode = "both")
tr_pipe = tfdet.dataset.pipeline.cast(tr_pipe, map = {"x_true":tf.float32, "y_true":tf.float32, "bbox_true":tf.float32, "mask_true":tf.float32})
tr_pipe = tr_pipe.batch(4).prefetch(1)

In [12]:
out = next(iter(tr_pipe))
image, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

if mask_true is None:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}".format(image.shape, y_true.shape, bbox_true.shape))
else:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}, mask_true:{3}".format(image.shape, y_true.shape, bbox_true.shape, mask_true.shape))

x_true:(4, 512, 512, 3), y_true:(4, 100, 1), bbox_true:(4, 100, 4), mask_true:(4, 100, 512, 512, 1)
